Aktuális könyvtárhelyzet visszaállítása (leállt tanulás folytatása/újraindítása esetén)

In [1]:
!cd /content/

Futtatással kapcsolatos konstansok

In [2]:
DATASET = '2009_old_2'
RUN_NAME = 'as4x200_sliced_large_finetune'
RUN_NUMBER = ''

DRIVE_PATH = '/content/drive/MyDrive/Colab_Notebooks/novenyfelismeres'

DATASET_PATH = f'/content/dataset/{DATASET}/data.yaml'
DATASET_NAME = '4x200_std'
MODEL_NAME = '/content/yolov5/models/yolov5m.yaml'
# MODEL_NAME = f'{DRIVE_PATH}/custom_models/yolov5l_transpose.yaml'

PROJECT_PATH = f'{DRIVE_PATH}/runs'

# PRETRAINED_WEIGHTS = '\'\''
PRETRAINED_WEIGHTS = 'yolov5m.pt'

EPOCHS = 200
BATCH_SIZE = 16

ONLY_VAL = True
CHECKPOINT = True

USE_ROBOFLOW = True

PATIENCE = 50

Modulok importálása, drive-hoz csatlakozás

In [3]:
from google.colab import drive
from google.colab import files
import zipfile
drive.mount('/content/drive')

Mounted at /content/drive


Yolo v5 setup

In [4]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 16088, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 16088 (delta 10), reused 14 (delta 2), pack-reused 16056
Receiving objects: 100% (16088/16088), 14.71 MiB | 16.42 MiB/s, done.
Resolving deltas: 100% (11038/11038), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.1/660.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.1.0 which is incompatible.


Adatok beimportálása (roboflow-al vagy saját zip-ből)

In [5]:
if USE_ROBOFLOW:
  !pip install roboflow

  from roboflow import Roboflow

  rf = Roboflow(api_key="eNnVdY8gmT6bwSVSovk6")

  version = 0
  if DATASET_NAME == '4x200_sliced':
    version = 1
  elif DATASET_NAME == '4x200_std':
    version = 2
  elif DATASET_NAME == 'as8x200':
    version = 3

  project = rf.workspace('thesis-rbwpo').project(DATASET_NAME)
  dataset = project.version(version).download('yolov5')

  DATASET_PATH = f'{dataset.location}/data.yaml'
else:
  zip_path = f'{DRIVE_PATH}/datasets/{DATASET}.zip'

  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
      zip_ref.extractall('/content/dataset')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstalle

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to 4x200_std-2 in yolov5pytorch:: 100%|██████████| 21138/21138 [00:06<00:00, 3068.28it/s]


Súly nélküli tanítás vagy (ha megadunk előre tanított súlyokat) továbbtanítás.

Megszakadt tanítást lehet folytatni a CHECKPOINT flag True-ra állítására.

In [6]:
proj_path = f'{PROJECT_PATH}/train'
if not ONLY_VAL:
  if CHECKPOINT:
    WEIGHT_PATH = f'{PROJECT_PATH}/train/{RUN_NAME}{RUN_NUMBER}/weights/last.pt'
    HYP_PATH = f'{PROJECT_PATH}/train/{RUN_NAME}{RUN_NUMBER}/hyp.yaml'
    !python train.py --resume $WEIGHT_PATH --img-size 640 --batch-size $BATCH_SIZE --data $DATASET_PATH --cfg $MODEL_NAME --name $RUN_NAME --hyp $HYP_PATH --weights $WEIGHT_PATH --project $proj_path
  else:
    !python train.py --img-size 640 --batch-size $BATCH_SIZE --epochs $EPOCHS --data $DATASET_PATH --patience $PATIENCE --weights $PRETRAINED_WEIGHTS --cfg $MODEL_NAME --name $RUN_NAME --project $proj_path

Validáció előtt a data.yaml-t rendbe kell tenni, mert néha rossz elérést ad a roboflow a teszthalmaznak

In [7]:
import yaml

with open(DATASET_PATH, 'r') as file:
    data = yaml.safe_load(file)

dataset_name = f'{data["roboflow"]["project"]}-{data["roboflow"]["version"]}'
data['test'] = f'{dataset_name}/test/images'

with open(DATASET_PATH, 'w') as file:
    yaml.dump(data, file)

Validáció

In [8]:
# run_num = str(int('1' if RUN_NUMBER == '' else RUN_NUMBER) + 1) if CHECKPOINT else ''
run_num = RUN_NUMBER
VAL_WEIGHT_PATH = f'{PROJECT_PATH}/train/{RUN_NAME}{run_num}/weights/best.pt'
proj_path = f'{PROJECT_PATH}/val'
!python val.py --img-size 640 --data $DATASET_PATH --task test --weights $VAL_WEIGHT_PATH --name $RUN_NAME --project $proj_path

val: data=/content/yolov5/4x200_std-2/data.yaml, weights=['/content/drive/MyDrive/Colab_Notebooks/novenyfelismeres/runs/train/as4x200_sliced_large_finetune/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/Colab_Notebooks/novenyfelismeres/runs/val, name=as4x200_sliced_large_finetune, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 267 layers, 46113663 parameters, 0 gradients, 107.7 GFLOPs
100% 755k/755k [00:00<00:00, 22.0MB/s]
test: Scanning /content/yolov5/4x200_std-2/test/labels... 80 images, 0 backgrounds, 0 corrupt: 100% 80/80 [00:00<00:00, 1156.10it/s]
test: New cache created: /content/yolov5/4x200_std-2/test/labels.cache
                 Class     Images